In [10]:
from solcx import compile_standard, install_solc

# Install Solidity compiler
install_solc("0.8.0")

# Compile the contract
with open("MockERC20.sol", "r") as file:
    contract_source = file.read()

compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {"MockERC20.sol": {"content": contract_source}},
    "settings": {
        "outputSelection": {
            "*": {"*": ["abi", "evm.bytecode"]}
        }
    }
})

# Extract ABI and bytecode
abi = compiled_sol["contracts"]["MockERC20.sol"]["MockERC20"]["abi"]
bytecode = compiled_sol["contracts"]["MockERC20.sol"]["MockERC20"]["evm"]["bytecode"]["object"]

In [11]:
print(abi)

[{'inputs': [{'internalType': 'uint256', 'name': '_initialSupply', 'type': 'uint256'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'owner', 'type': 'address'}, {'indexed': True, 'internalType': 'address', 'name': 'spender', 'type': 'address'}, {'indexed': False, 'internalType': 'uint256', 'name': 'value', 'type': 'uint256'}], 'name': 'Approval', 'type': 'event'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'from', 'type': 'address'}, {'indexed': True, 'internalType': 'address', 'name': 'to', 'type': 'address'}, {'indexed': False, 'internalType': 'uint256', 'name': 'value', 'type': 'uint256'}], 'name': 'Transfer', 'type': 'event'}, {'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}, {'internalType': 'address', 'name': '', 'type': 'address'}], 'name': 'allowance', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint25

In [12]:
print(bytecode)

60806040526040518060400160405280600981526020017f4d6f636b455243323000000000000000000000000000000000000000000000008152506000908051906020019062000051929190620001da565b506040518060400160405280600481526020017f4d4f434b00000000000000000000000000000000000000000000000000000000815250600190805190602001906200009f929190620001da565b506012600260006101000a81548160ff021916908360ff160217905550348015620000c957600080fd5b506040516200175e3803806200175e8339818101604052810190620000ef9190620002a1565b600260009054906101000a900460ff1660ff16600a62000110919062000356565b816200011d919062000493565b600381905550600354600460003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020819055503373ffffffffffffffffffffffffffffffffffffffff16600073ffffffffffffffffffffffffffffffffffffffff167fddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef600354604051620001cb9190620002de565b60405180910390a350620005b9565b828054620001e890620004fe565b9060005260206000209060

In [13]:
from web3 import Web3

# Connect to Ganache
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check if connected
if not web3.is_connected():
    raise Exception("Failed to connect to Ganache")

# Set up account
account_address = "0xAC74e81CC49A35556E2C416accCb7BDE464927D7"  # Replace with your Ganache account address
private_key = "0x7d4534266184e1bb56fd7e11b32237e2bccda04af36c29cd2daf0598a0cefbde"  # Replace with your Ganache private key

# Contract ABI and bytecode (from Step 4)
# abi = [...]  # Replace with the ABI from compilation
# bytecode = "0x..."  # Replace with the bytecode from compilation

# Create contract instance
MockERC20 = web3.eth.contract(abi=abi, bytecode=bytecode)

# Build transaction
initial_supply = 1000000  # 1 million tokens
transaction = MockERC20.constructor(initial_supply).build_transaction({
    "chainId": 1337,  # Confirm chain ID in Ganache
    "gas": 2000000,
    "gasPrice": web3.to_wei("20", "gwei"),
    "nonce": web3.eth.get_transaction_count(account_address),
    "from": account_address,  # Required in web3.py v6+
})

# Sign and send transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key)
tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Wait for receipt
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Contract deployed at: {tx_receipt.contractAddress}")

Contract deployed at: 0x91DC50bf2dA9D22F2026708F6A93D29155bFcC98


In [ ]:
from web3 import Web3
from solcx import compile_source

# Connect to Ganache
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

# ERC20 Token Contract
erc20_source =
    '''
pragma solidity ^0.8.0;
contract MockERC20 {
    string public name = "MockToken";
    string public symbol = "MTK";
    uint8 public decimals = 18;
    uint256 public totalSupply = 1000000 * 10**18;
    mapping(address => uint256) public balanceOf;

    constructor() {
        balanceOf[msg.sender] = totalSupply;
    }

    function transfer(address to, uint256 value) public returns (bool) {
        require(balanceOf[msg.sender] >= value, "Insufficient balance");
        balanceOf[msg.sender] -= value;
        balanceOf[to] += value;
        return true;
    }
}
'''

# Compile and deploy
compiled = compile_source(erc20_source)
contract_id = compiled.popitem()[0]
erc20_contract = w3.eth.contract(
    abi=compiled[contract_id]['abi'],
    bytecode=compiled[contract_id]['bin']
)
tx_hash = erc20_contract.constructor().transact({'from': w3.eth.accounts[0]})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
erc20_address = tx_receipt['contractAddress']
